In [ ]:
# %pip install -r ./requirements.txt

In [1]:
%pip install bitsandbytes accelerate peft unbabel-comet

Defaulting to user installation because normal site-packages is not writeable
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.17.9 requires pyarrow<10.0dev,>=3.0.0, but you have pyarrow 17.0.0 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
# %pip uninstall protobuf -y
%pip install protobuf==3.20

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unbabel-comet 2.2.2 requires protobuf<5.0.0,>=4.24.4, but you have protobuf 3.20.0 which is incompatible.
google-api-core 2.11.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
google-cloud-bigquery 3.10.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
google-cloud-bigquery-con

In [ ]:
import os
import random
import torch
import numpy as np
from typing import Dict, List
from tqdm import tqdm

from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    BitsAndBytesConfig
)

from peft import (
    prepare_model_for_kbit_training
)

from comet import (
    download_model, 
    load_from_checkpoint
)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-10-16 22:16:30.085625: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-16 22:16:34.500857: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
def seed_everything(seed):
    # Fix seeds
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything(123456)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

'cuda'

In [11]:
dataset = load_dataset(
    "json",
    data_files={
        "train": "/home/jupyter/datasphere/project/data/flores200_dev/en_uz_dev.jsonl",
        "val": "/home/jupyter/datasphere/project/data/flores200_devtest/en_uz_devtest.jsonl",
    },
)

In [12]:
model_path = "haoranxu/X-ALMA-13B-Pretrain"
tokenizer = AutoTokenizer.from_pretrained(
    model_path, model_max_length=256, padding_side="left"
)
# tokenizer.pad_token = tokenizer.eos_token  # Set padding token as EOS token
# tokenizer.pad_token_id = tokenizer.eos_token_id  # Set padding token as EOS token
# tokenizer.pad_token = tokenizer.bos_token  # Set padding token as BOS token
# tokenizer.pad_token_id = tokenizer.bos_token_id  # Set padding token as BOS token

In [13]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [14]:
model = AutoModelForCausalLM.from_pretrained(model_path, quantization_config=bnb_config)
model = prepare_model_for_kbit_training(model)

`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 6/6 [06:59<00:00, 70.00s/it]


In [15]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((5120,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNor

In [16]:
def tokenize(samples):
    inputs = [
        f"Translate this from Uzbek to English:\nUzbek: {uz}\nEnglish:"
        for uz in samples["uz"]
    ]
    targets = samples["en"]

    model_inputs = tokenizer(
        inputs,
        truncation=True,
        max_length=256,
        padding="max_length",
        return_tensors="pt",
    )
    labels = tokenizer(
        targets,
        truncation=True,
        max_length=256,
        padding="max_length",
        return_tensors="pt",
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
tokenized_train_dataset = dataset["train"].map(tokenize, batched=True)
tokenized_val_dataset = dataset["val"].map(tokenize, batched=True)

Map: 100%|██████████| 1012/1012 [00:00<00:00, 3825.01 examples/s]


In [19]:
tokenizer.decode(tokenized_train_dataset["input_ids"][0])

"<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><s> Translate this from Uzbek to English:\nUzbek: Dushanba kuni Stenford Universitetining Tibbiyot maktabi olimlari hujayralarni turlariga qarab saralay oladigan yangi tashxis vositasi ixtirosini e'lon qildi: har biri taxminan bir AQSH senti atrofida bo'lgan standart rangli printerlardan foydalangan holda ishlab chiqarish mumkin bo'lgan ingichka bosma chip.\nEnglish:"

In [20]:
eval_prompt = "Translate this from Uzbek to English:\nUzbek: Dushanba kuni Stenford Universitetining Tibbiyot maktabi olimlari hujayralarni turlariga qarab saralay oladigan yangi tashxis vositasi ixtirosini e'lon qildi: har biri taxminan bir AQSH senti atrofida bo'lgan standart rangli printerlardan foydalangan holda ishlab chiqarish mumkin bo'lgan ingichka bosma chip.\nEnglish:"

In [21]:
model_input = tokenizer(eval_prompt, return_tensors="pt").to(device)
model.eval()
with torch.no_grad():
    print(
        tokenizer.decode(
            model.generate(**model_input, max_new_tokens=256, pad_token_id=2)[0],
            skip_special_tokens=True,
        )
    )

print(tokenized_train_dataset["labels"][0])

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Translate this from Uzbek to English:
Uzbek: Dushanba kuni Stenford Universitetining Tibbiyot maktabi olimlari hujayralarni turlariga qarab saralay oladigan yangi tashxis vositasi ixtirosini e'lon qildi: har biri taxminan bir AQSH senti atrofida bo'lgan standart rangli printerlardan foydalangan holda ishlab chiqarish mumkin bo'lgan ingichka bosma chip.
English: On Monday, researchers at Stanford University's School of Medicine announced a new diagnostic tool that can determine the type of cancer a patient has by analyzing the cells that have been shed from the tumor and collected on a standard blue plastic sheet.
The test, called Cellular Spotting Device, or CSD, uses a small chip that can be inserted into a standard blue plastic sheet. The chip, which costs about $1, can be used to detect cancer by looking for the presence of a specific type of cell called a tumor-associated macrophage, or TAM.
The chip is designed to be used with a standard blue plastic sheet that is placed on a micr

In [19]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((5120,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNor

In [18]:
model_path_comet = download_model("Unbabel/wmt22-comet-da")
model_comet = load_from_checkpoint(model_path_comet)
model_comet = model_comet.to(device)


def comet(data: List[Dict[str, str]]) -> List[float]:
    '''Format
    data = [
    {
        # Source, текст, который надо перевести, src
        "src": "В понедельник", 
        
        # Machine Translation
        "mt": "On Monday", 
        
        # Эталонный перевод, en
        "ref": "On Monday" 
    }'''

    comet_metric = model_comet.predict(data, batch_size=8, gpus=1)
    return comet_metric.scores

Fetching 5 files: 100%|██████████| 5/5 [00:20<00:00,  4.01s/it]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../tmp/xdg_cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/371e9839ca4e213dde891b066cf3080f75ec7e72/checkpoints/model.ckpt`
Encoder model frozen.
/home/jupyter/.local/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [ ]:
data = [
    {
        "src": "Dem Feuer konnte Einhalt geboten werden",
        "mt": "The fire could be stopped",
        "ref": "They were able to control the fire."
    },
    {
        "src": "Schulen und Kindergärten wurden eröffnet.",
        "mt": "Schools and kindergartens were open",
        "ref": "Schools and kindergartens opened"
    }
]
model_output = model_comet.predict(data, batch_size=8, gpus=1)
print(model_output)

In [20]:
all_predictions = []
all_references = []
all_sources = []

batch_size = 1  # in Kaggle
# batch_size = 4    # in DataSphere

val_dataset = tokenized_val_dataset.shuffle().select(range(100))

model.eval()

for i in tqdm(range(0, len(val_dataset), batch_size)):
    batch = val_dataset.select(range(i, min(i + batch_size, len(val_dataset))))
    input_ids = torch.tensor(batch["input_ids"]).to("cuda")

    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=input_ids,
            num_beams=5,
            max_new_tokens=200,
            no_repeat_ngram_size=3,
            early_stopping=True,
        )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    for uz, en, translated in zip(batch["uz"], batch["en"], outputs):
        all_sources.append(uz)
        all_references.append(en)
        all_predictions.append(translated.split('English:')[-1].strip())

100%|██████████| 100/100 [52:28<00:00, 31.49s/it]


In [21]:
def evaluate(model):
    model.eval()

    comet_data = [
        {"src": src, "mt": pred, "ref": ref}
        for src, pred, ref in zip(all_sources, all_predictions, all_references)
    ]

    comet_scores = comet(comet_data)
    avg_comet_score = sum(comet_scores) / len(comet_scores)

    return avg_comet_score, all_predictions, all_references


avg_comet_score, predictions, references = evaluate(model, tokenizer)
print(f"Average COMET score: {avg_comet_score:.4f}")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable

Average COMET score: 0.4768


In [ ]:
print(tokenizer.decode(tokenized_train_dataset["labels"][0]))